In [4]:
#export
import pickle, gzip, math, os, time, shutil, torch, matplotlib as mpl, numpy as np
import pandas as pd
from pathlib import Path
from torch import nn
import torch

from torch.utils.data import DataLoader, default_collate
from typing import Mapping, Tuple, Sequence, Union, TypeVar, List, Callable

from miniai.dataset import *
from datasets import Dataset
from torch.utils.data import DataLoader
import datasets



In [6]:
# download mnist from hugginface datasets
fashion_mnist = datasets.load_dataset('fashion_mnist')

Found cached dataset fashion_mnist (/Users/diegomedina-bernal/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)
100%|██████████| 2/2 [00:00<00:00, 144.90it/s]


In [7]:

fashion_mnist

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [ ]:
x, y = "image", "label"